# GroupfMRI6a - Use motor rule regions to predict Motor response regions
#### Using ActFlow

#### Takuya Ito
#### 03/01/2019

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group
import nibabel as nib
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


/home/tito/miniconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 1.0 Define functions for motor response decodings

#### Load in FC mappings

## 1.01 Load data for RH and LH responses

In [11]:
tools_group = reload(tools_group)
inputtype = 'pitch'
data_task_rh = np.zeros((len(glasser2),2,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),2,len(subjNums)))
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)

# Create arrays for visualization
layerByLayerActivityPredictions_rmid = np.zeros((len(glasser2),7))
layerByLayerActivityPredictions_rind = np.zeros((len(glasser2),7))

# Identify target indices
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))
        
target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)

scount = 0
for subj in subjNums:
    print 'Running compositional analysis on subject', scount+1, '/', len(subjNums)
    tmp1, tmp2, tmp3, tmp4, tmp5, tmp6, input_ind, sr_ind, motorrule_ind = tools_group.compositionalActFlow(subj, inputtype)
    data_input = tmp1
    data_sr = tmp2
    data_motorrule = tmp3
    fc_input2sr = tmp4
    fc_sr2motorrule = tmp5
    fc_motorrule2motorresp = tmp6
    
    data_task_rh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Right')
    data_task_lh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Left')
    
    #####################
    # Run composition activity flow
    
    ## Run for RH
    #### Input to SR regions
    # first identify non-overlapping indices
    unique_input_ind = np.where(np.in1d(input_ind,sr_ind)==False)[0]
    actflow_sr_rmid = np.dot(data_input[unique_input_ind,0],fc_input2sr) # rmid - high high stim
    actflow_sr_rind = np.dot(data_input[unique_input_ind,3],fc_input2sr) # rind - low low stim
    # Construct composition sr representations
    unique_sr_ind = np.where(np.in1d(sr_ind,motorrule_ind)==False)[0]
    sr_composition_rmid = np.multiply(actflow_sr_rmid, data_sr[:,3]) # Neither 
    sr_composition_rind = np.multiply(actflow_sr_rind, data_sr[:,3]) # Neither 
    actflow_motorrule_rmid = np.dot(sr_composition_rmid[unique_sr_ind],fc_sr2motorrule)
    actflow_motorrule_rind = np.dot(sr_composition_rind[unique_sr_ind],fc_sr2motorrule)
    # Construct motor rule compositions
    unique_motor_ind = np.where(np.in1d(motorrule_ind,target_ind)==False)[0]
    motorrule_composition_rmid = np.multiply(actflow_motorrule_rmid, data_motorrule[:,3]) # This is rind - logic gates force rmid
    motorrule_composition_rind = np.multiply(actflow_motorrule_rind, data_motorrule[:,3]) # This is rind
    actflow_rh[target_ind,0,scount] = np.dot(motorrule_composition_rmid[unique_motor_ind],fc_motorrule2motorresp)
    actflow_rh[target_ind,1,scount] = np.dot(motorrule_composition_rind[unique_motor_ind],fc_motorrule2motorresp)
    ## Create arrays for visualization purposes
    # input
    layerByLayerActivityPredictions_rmid[input_ind,0] = layerByLayerActivityPredictions_rmid[input_ind,0] + data_input[:,0]
    layerByLayerActivityPredictions_rind[input_ind,0] = layerByLayerActivityPredictions_rind[input_ind,0] + data_input[:,3]
    # sr prediction
    layerByLayerActivityPredictions_rmid[sr_ind,1] = layerByLayerActivityPredictions_rmid[sr_ind,1] + actflow_sr_rmid
    layerByLayerActivityPredictions_rind[sr_ind,1] = layerByLayerActivityPredictions_rind[sr_ind,1] + actflow_sr_rind
    # sr composition
    layerByLayerActivityPredictions_rmid[sr_ind,2] = layerByLayerActivityPredictions_rmid[sr_ind,2] + sr_composition_rmid
    layerByLayerActivityPredictions_rind[sr_ind,2] = layerByLayerActivityPredictions_rind[sr_ind,2] + sr_composition_rind
    # motor rule prediction
    layerByLayerActivityPredictions_rmid[motorrule_ind,3] = layerByLayerActivityPredictions_rmid[motorrule_ind,3] + actflow_motorrule_rmid
    layerByLayerActivityPredictions_rind[motorrule_ind,3] = layerByLayerActivityPredictions_rind[motorrule_ind,3] + actflow_motorrule_rind
    # motor rule composition
    layerByLayerActivityPredictions_rmid[motorrule_ind,4] = layerByLayerActivityPredictions_rmid[motorrule_ind,4] + motorrule_composition_rmid
    layerByLayerActivityPredictions_rind[motorrule_ind,4] = layerByLayerActivityPredictions_rind[motorrule_ind,4] + motorrule_composition_rind
    # motor resp actflow
    layerByLayerActivityPredictions_rmid[target_ind,5] = layerByLayerActivityPredictions_rmid[target_ind,5] + actflow_rh[target_ind,0,scount]
    layerByLayerActivityPredictions_rind[target_ind,5] = layerByLayerActivityPredictions_rind[target_ind,5] + actflow_rh[target_ind,1,scount]
    # motor resp real
    layerByLayerActivityPredictions_rmid[target_ind,6] = layerByLayerActivityPredictions_rmid[target_ind,6] + data_task_rh[target_ind,0,scount]
    layerByLayerActivityPredictions_rind[target_ind,6] = layerByLayerActivityPredictions_rind[target_ind,6] + data_task_rh[target_ind,1,scount]
    
    
    ## Run for LH
    #### Input to SR regions
    # first identify non-overlapping indices
    unique_input_ind = np.where(np.in1d(input_ind,sr_ind)==False)[0]
    actflow_sr_lmid = np.dot(data_input[unique_input_ind,0],fc_input2sr) # lmid - vert vert stim
    actflow_sr_lind = np.dot(data_input[unique_input_ind,3],fc_input2sr) # lind - horiz horiz stim
    # Construct composition sr representations
    unique_sr_ind = np.where(np.in1d(sr_ind,motorrule_ind)==False)[0]
    sr_composition_lmid = np.multiply(actflow_sr_lmid, data_sr[:,3]) # Neither 
    sr_composition_lind = np.multiply(actflow_sr_lind, data_sr[:,3]) # Neither 
    actflow_motorrule_lmid = np.dot(sr_composition_lmid[unique_sr_ind],fc_sr2motorrule)
    actflow_motorrule_lind = np.dot(sr_composition_lind[unique_sr_ind],fc_sr2motorrule)
    # Construct motor rule compositions
    unique_motor_ind = np.where(np.in1d(motorrule_ind,target_ind)==False)[0]
    motorrule_composition_lmid = np.multiply(actflow_motorrule_lmid, data_motorrule[:,1])
    motorrule_composition_lind = np.multiply(actflow_motorrule_lind, data_motorrule[:,1])
    actflow_lh[target_ind,0,scount] = np.dot(motorrule_composition_lmid[unique_motor_ind],fc_motorrule2motorresp)
    actflow_lh[target_ind,1,scount] = np.dot(motorrule_composition_lind[unique_motor_ind],fc_motorrule2motorresp)
    #####################
    
    scount += 1
    
# Print out to file
layerByLayerActivityPredictions_rmid = np.divide(layerByLayerActivityPredictions_rmid,float(len(subjNums)))
layerByLayerActivityPredictions_rind = np.divide(layerByLayerActivityPredictions_rind,float(len(subjNums)))
tools_group.mapBackToSurface(layerByLayerActivityPredictions_rmid,'10bLayerByLayerPredictions_stimHIGHHIGH_NEITHER_HIGH_RIND_RespRMID')
tools_group.mapBackToSurface(layerByLayerActivityPredictions_rind,'10bLayerByLayerPredictions_stimLOWLOW_NEITHER_HIGH_RMID_RespRIND')
del fc_input2sr, fc_motorrule2motorresp, fc_sr2motorrule

Running compositional analysis on subject 1 / 96
Running compositional analysis on subject 2 / 96
Running compositional analysis on subject 3 / 96
Running compositional analysis on subject 4 / 96
Running compositional analysis on subject 5 / 96
Running compositional analysis on subject 6 / 96
Running compositional analysis on subject 7 / 96
Running compositional analysis on subject 8 / 96
Running compositional analysis on subject 9 / 96
Running compositional analysis on subject 10 / 96
Running compositional analysis on subject 11 / 96
Running compositional analysis on subject 12 / 96
Running compositional analysis on subject 13 / 96
Running compositional analysis on subject 14 / 96
Running compositional analysis on subject 15 / 96
Running compositional analysis on subject 16 / 96
Running compositional analysis on subject 17 / 96
Running compositional analysis on subject 18 / 96
Running compositional analysis on subject 19 / 96
Running compositional analysis on subject 20 / 96
Running c

## 1.1 Run across subject decoding on right-hand motor responses

In [12]:
tools_group = reload(tools_group)

In [13]:
nproc = 20
ncvs = 1
nResponses = 2

# realdata = stats.zscore(data_task_rh[target_ind,:,:],axis=0).copy()
# flowata = stats.zscore(actflow_data[target_ind,:,:],axis=0).copy()
realdata = data_task_rh[target_ind,:,:].copy()
flowdata = actflow_rh[target_ind,:,:].copy()


distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:] = tools_group.actflowDecodings(realdata,
                                                          flowdata,
                                                          ncvs=ncvs, nproc=nproc)

# distances_baseline_rh[0,:] = tools_group.actflowDecodings(realdata,
#                                                           data_composition_rh[target_ind,:,:],
#                                                           ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [14]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print 'Activity flow accuracy =', statistics_rh[0,0]
print 'p =', statistics_rh[0,1]



Activity flow accuracy = 0.739583333333
p = 1.05300690701e-11


# 2.0 Run across subject decoding on left-hand motor responses

In [15]:
tools_group = reload(tools_group)

In [16]:
nproc = 20
ncvs = 1

# realdata = stats.zscore(data_task_lh[target_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_lh[target_ind,:,:],axis=0).copy()
realdata = data_task_lh[target_ind,:,:].copy()
flowdata = actflow_lh[target_ind,:,:].copy()


distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:] = tools_group.actflowDecodings(realdata,
                                                          flowdata,
                                                          ncvs=ncvs, nproc=nproc)

# distances_baseline_lh[0,:] = tools_group.actflowDecodings(realdata,
#                                                           data_composition_lh[target_ind,:,:],
#                                                           ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [17]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print 'Activity flow accuracy =', statistics_lh[0,0]
print 'p =', statistics_lh[0,1]



Activity flow accuracy = 0.625
p = 0.000327515174901
